## Install requirements

In [ ]:
!python.exe -m pip install --upgrade pip

In [ ]:
%pip install -r requirements-torch-windows.txt -U
%pip install -r requirements.txt -U

## Init stable diffusion

In [ ]:
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import torch

def initdiffusionpipeline(model_id):
    # Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    pipe = pipe.to("cuda")
    return pipe

#diffusionpipeline=initdiffusionpipeline("stabilityai/stable-diffusion-2-1")
#diffusionpipeline=initdiffusionpipeline("dreamlike-art/dreamlike-diffusion-1.0")
diffusionpipeline=initdiffusionpipeline("runwayml/stable-diffusion-v1-5")


In [ ]:
import re
def saveimage(image, prompt, seed, postfix=""):
    cleanprompt = re.sub("( |,)", '-', prompt)
    fname = f"images/{cleanprompt}-{seed}-{postfix}.png"
    image.save(fname)
    print(fname)
    display(image)
    #image.show() # open outside

In [ ]:
def generate(prompt,nbimages=1,seed=42):
    images = []
    for i in range(nbimages):
        generator = torch.Generator("cuda").manual_seed(seed+i)
        image = diffusionpipeline(prompt,num_images_per_prompt=1, generator=generator).images[0]
        saveimage(image, prompt, seed+i)
        images.append(image)
    #images = diffusionpipeline(prompt,num_images_per_prompt=nbimages, generator=generator).images
    return images

In [ ]:
def expand(metaprompt):
    promptlist=[""]
    for part in metaprompt.split("/"):
        newpromptlist=[]
        for variation in part.split("|"):
            for partialprompt in promptlist:
                newpromptlist.append(partialprompt+variation) # combine
        promptlist=newpromptlist.copy() #ensure its a copy
    return promptlist

## Run

In [ ]:
prompt="photo of a /kraken|octopus|creature/ lurking in the deep green blue sea, deep blue shadows and sun rays, undersea jungle, high definition rendering"
prompts=expand(prompt)
for p in prompts:
    print(p)
    generate(p)
    